In [1]:
with open("key.txt", "r") as file:
    API_KEY_CLARIN = file.read().strip()

MODELS_ENDPOINT = "https://services.clarin-pl.eu/api/v1/oapi/models"
COMPLETIONS_ENDPOINT = "https://services.clarin-pl.eu/api/v1/oapi/chat/completions"
CLARIN_BASE_URL = "https://services.clarin-pl.eu/api/v1/oapi"

MODEL_ID = "bielik"

In [2]:
import requests
import pandas as pd
import json
import time
import requests

In [3]:
def prompt_chat_custom_temperature(model_id, prompt, temperature = 0.3):
    url = COMPLETIONS_ENDPOINT
    headers = {"Authorization": f"Bearer {API_KEY_CLARIN}", "Content-Type": "application/json"}
    messages = []
    messages.append({"role": "user", "content": prompt})
    data = {
        "model": model_id,
        "messages": messages,
        "temperature": temperature
        #"max_tokens": max_tokens
    }
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        try:
            data = response.json()
            if isinstance(data, dict) and "choices" in data and data["choices"]:
                return data["choices"][0].get("message", {}).get("content", None)
            else:
                return None  # Handle unexpected response structure
        except (ValueError, TypeError, AttributeError):
            return None  # Handle cases where response.json() is None or not as expected
    else:
        return None  # Handle non-200 status codes

In [4]:
df = pd.read_csv("data/dataset.csv", delimiter="|")
df

,en_text,pl_text
0,"Hey, everyone! Wanna hear about Fyresdal?","Ej, ludzie! Chcecie posłuchać o Fyresdal?"
1,"Hey, good folk! Wanna hear what our brave guar...","Ej, ludzie! Chcecie usłyszeć, co nasz dzielny ..."
2,"Shut your trap, dammit!","Stul pysk, do cholery!"
3,Four against one - and you came out alive.,Czterech na jednego. A dałeś radę.
4,Shame to let such a lad waste away behind bars...,"Szkoda, żeby taki chłop marniał za kratkami......"
...,...,...
8375,Farewell.,Bywaj.
8376,Farewell.,Bywaj.
8377,What do you have on offer?,Pokaż mi swoje towary.
8378,Show me what you have in stock.,Pokaż mi swoje towary.


In [5]:
sentences = ["Unless… She coexists with drowners… Maybe I could make an extract from their glands, cover up my scent.", 
             "When'd you lose your voice? Can you show me, hold up some fingers?",
             "Pirate Leader",
             "You're dull as dirt. *Yaaawn*",
             "Jarl of Undvik - tell me about his death.",
             "Let's go see Udalryk. Be interesting to see if there actually is a ghost in all this.",
             "Bad way to play it out.",
             "So? Ready?"]

In [7]:
prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: '{sentences[3]}'", 
    0.0
)

'Jesteś nudny jak flaki z olejem. *Ziewnięcie*'

In [8]:
context = "Zdanie jest elementem dialogu. Wypowiada je mężczyzna w stronę kobiety. \n"

In [9]:
prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Oto kontekst: {context} Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: '{sentences[3]}'", 
    0.0
)

'Jesteś nudna jak flaki z olejem. *Ziewnięcie*'

In [12]:
rule1 = "W dialogu mogą wystąpić słowa opisujące dzwięki wydawane przez osobę, która mówi. Podczas tłumaczenia zamień te słowa na faktycznie dzwięki, wydawane podczas tej czynności. \n" 
examples = "Yawn - Aaah \n Sneeze - Apsik \n"

In [13]:
prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Zasada: {rule1} Przykłady: {examples} Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: '{sentences[3]}'", 
    0.0
)

'Jesteś nudny jak flaki z olejem. *Aaah*'

In [14]:
prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Oto kontekst: {context} Zasada: {rule1} Przykłady: {examples} Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: '{sentences[3]}'", 
    0.0
)

'Jesteś nudny jak flaki z olejem. *Aaah*'

In [15]:
prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Oto podane zdanie: {sentences[3]} \n Kontekst: {context} Zasada: {rule1} Przykłady: {examples} Przetłumacz podane zdanie na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie. Podczas tłumaczenia uwzględnij zasady i kontekst:", 
    0.0
)

'Jesteś nudna jak flaki z olejem. *Ziewnięcie*'

In [16]:
translationv1 = prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: '{sentences[3]}'", 
    0.0
)
translationv1

'Jesteś nudny jak flaki z olejem. *Ziewnięcie*'

In [17]:
translationv2 = prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Oto kontekst: {context} Przepisz podane zdanie na polski zgodnie z podanym kotekstem i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: '{translationv1}'",
    0.0
)
translationv2

'"Jesteś tak nudny, że mogłabym zasnąć stojąc." *Ziewnięcie*'

In [18]:
context2 = "Zdanie wypowiadane jest przez mężczyzne w stronę kobiety \n"

In [19]:
translationv2 = prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Zdanie po tłumaczniu bez kontekstu: '{translationv1}'\n Kontekst zdania: {context}\n Odmień wyrazy w podanym tłumaczeniu tak aby pasowały do kontekstu zdania. Napisz tylko i wyłącznie samo poprawione tłumaczenie i nie dodawaj żadnych dodatkowych wyujaśnień:",
    0.0
)
translationv2

'Jesteś nudna jak flaki z olejem. *Ziewnięcie*'

In [20]:
translationv3 = prompt_chat_custom_temperature(
    MODEL_ID, 
    f"Zdanie po tłumaczniu '{translationv2}'\n Zasady: {rule1}\n Przykłady zasady: {examples}\n Zastosuj zasady wedle przykładów na podanym zdaniu i napisz tylko i wyłącznie samo poprawione zdanie, nie dodawaj żadnych dodatkowych wyjaśnień:",
    0.0
)
translationv3

'Jesteś nudna jak flaki z olejem. *Aaah*'